<h1>Outline</h1>

 * <h2 style="color:green">Let's talk about AI today</h2> 
 * <h2 style="color:green">Privacy vs Convenience</h2> 
 * <h2 style="color:green">Overview of Recommender Systems</h2> 
 * <h2 style="color:green">Methods for Building a Recommender System</h2> 
 * <h2 style="color:red">Implementing a Recommender System</h2> 
 





On that note, we end the overview and begin our workshop building a simple recommendation Engine. 

#### Instructions

* Go to https://go.ncsu.edu/flames_ml
* Download the repository 
* Go to https://cocalc.com/
* Create a new project called Flames_Recommender_systems
* Click "Upload File", Choose "twenty_users_ratings.csv" from Highschool Folder on your computer
* Click "Upload File", Choose "Flames_Recommenders_System_Code.ipynb" from Highschool Folder on your computer
* Open "Collaborative_Filtering_Code.ipynb"
* Click run all

#### Section 1: Collaborative Filtering with Python

<h2><p style="display: block; text-align: center;">Think about Clusters, We think Similarities</p></h2>





We'll be building a collaborative filtering based recommendation systems. That calculates similarities between ratings users have given a movie and use used a similarity matrics called euclidean similarity, to find similar users and recommend movies from those similar users. 

* If John loves it and John is similar to Mary, it's likely that Mary will love it too!

User-based Filtering: these system will recommend movies to a user that similar users have liked. For example, let's say John and Mary have watched and liked a set of movies in the past, that means they are similar. We'll give a score to the extent to which they are similar using Euclidean similarity matrics. (that is, they largely like the same movies). 

Now, suppose some new movies have been released on Netflix. If John has watched and loved it. Then our recommedation system will suggest these set of new movies to Mary as it is highly likely that Mary will like it too.

#### Question

* From what we know about collaborative filtering, can anyone tell us what data/attributes we need for collaborative filtering?

#### Answer
* Movies all users have watched 
* Ratings by all users on all they movies they have watched

### Let's Code

The code used in this section was adapted from: [ai_society](https://ai-society.github.io/Collaborative-Filtering/)
You can also modify it for other datasets as well.


In [9]:
#Import necessary libraries
import math
import pandas as pd

In [10]:
#Let's read our data into memory
ratings = pd.read_csv("twenty_users_ratings.csv",  index_col=False)
ratings.tail(10)


,userId,movieId,rating,timestamp,movie_title,user_name
2129,20,2115,3.5,1126539606,Indiana Jones and the Temple of Doom (1984),Sarah
2130,20,2571,4.0,1126539826,"Matrix, The (1999)",Sarah
2131,20,2710,4.0,1126539586,"Blair Witch Project, The (1999)",Sarah
2132,20,2987,3.0,1126539549,Who Framed Roger Rabbit? (1988),Sarah
2133,20,3176,3.0,1126539815,"Talented Mr. Ripley, The (1999)",Sarah
2134,20,3785,3.0,1126539891,Scary Movie (2000),Sarah
2135,20,4306,3.5,1126539543,Shrek (2001),Sarah
2136,20,4351,3.5,1126539896,Point Break (1991),Sarah
2137,20,5952,3.5,1126539610,"Lord of the Rings: The Two Towers, The (2002)",Sarah
2138,20,6807,3.5,1126539870,Monty Python's The Meaning of Life (1983),Sarah


In [11]:
#Let's see all the users 
ratings.user_name.unique()

array(['Michael', 'Christopher', 'Jessica', 'Matthew', 'Ashley',
       'Jennifer', 'Joshua', 'Amanda', 'Daniel', 'David', 'James',
       'Robert', 'John', 'Joseph', 'Andrew', 'Ryan', 'Brandon', 'Jason',
       'Justin', 'Sarah'], dtype=object)

In [12]:
# Make a copy of our dataframe so that we don't accidentally modify it
ratings_dict = ratings.copy()

In [13]:
#Make a dictionary of users and the movies they rated
all_dict = dict()
for usr in ratings_dict.user_name:
    one_usr = ratings_dict[ratings_dict.user_name==usr]
    usr_dict = dict(zip(one_usr.movie_title, one_usr.rating))
    #all_dict = {usr: usr_dict}
    all_dict[str(usr)] = usr_dict
    


In [14]:
data = all_dict
#Function for calculating similarities
def euclidean_similarity(person1, person2):

    common_ranked_items = [itm for itm in data[person1] if itm in data[person2]]
    rankings = [(data[person1][itm], data[person2][itm]) for itm in common_ranked_items]
    distance = [pow(rank[0] - rank[1], 2) for rank in rankings]

    return 1 / (1 + sum(distance))

In [15]:
# This is a recommend function, which we call to recommend movies to users
def recommend(person, bound, similarity=euclidean_similarity):
    scores = [(similarity(person, other), other) for other in data if other != person]
    
    scores.sort()
    scores.reverse()
    scores = scores[0:bound]

    print (scores)
    #print (scores.) #(0.5, 'Jennifer'), (0.4, 'Robert'), (0.3076923076923077, 'Brandon'),... 0.5 is more similar

    recomms = {}

    for sim, other in scores:
        ranked = data[other] #Get the movies and respective rating from these users in the same cluster

        for itm in ranked: #for all movie similar users have watched
            if itm not in data[person]: #get movies this user has not already watched
                weight = sim * ranked[itm] #multiply the similarity by the 

                if itm in recomms:
                    s, weights = recomms[itm]
                    recomms[itm] = (s + sim, weights + [weight])
                else:
                    recomms[itm] = (sim, [weight])

    #print(recomms)               
    for r in recomms:
        sim, item = recomms[r]
        #print(item) 
        #print(sum(item)) 
        recomms[r] = sum(item) / sim #predicted ranking for each movie = actual ranking/similarity

    #return recomms
    return sorted(recomms.items(), key=lambda x: x[1], reverse=True)[0:bound]

In [16]:
# Finally, Let's recommend 5 movies to Michael
recommend("Michael", 5, euclidean_similarity)

[(0.5, 'Jennifer'), (0.4, 'Robert'), (0.3076923076923077, 'Brandon'), (0.2857142857142857, 'David'), (0.2857142857142857, 'Andrew')]


[('Mighty Aphrodite (1995)', 5.0),
 ("Mr. Holland's Opus (1995)", 5.0),
 ('Fatal Attraction (1987)', 5.0),
 ('Patriot Games (1992)', 5.0),
 ('Gladiator (2000)', 5.0)]

<h1>Thanks for coming & Happy Recommending!</h1>